Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

In [1]:
import pandas as pd 
import numpy as np 

from sklearn.pipeline import Pipeline

import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier 
from catboost import CatBoostClassifier

from sklearn.metrics import f1_score

import time
import warnings
warnings.filterwarnings('ignore')

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

import plotly.graph_objects as go

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
df.tail()

,text,toxic
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0
159570,"""\nAnd ... I really don't think you understand...",0


In [6]:
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

**Первичный анализ:**

- 159571 строка
- Дубликатов и пропусков нет
- 2 столбца: toxic, text
- В столбце text содержатся тексты твитов
- В столбе toxic булевые значения является ли данный твит токсичным или нет
- 90% твитов не токсичны

In [7]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [8]:
%%time
df['text'] = df['text'].map(lambda x: clean_text(x))

CPU times: user 12.6 s, sys: 158 ms, total: 12.8 s
Wall time: 12.8 s


С помощью регулярных выражений обработали текст для дальнейшего исследования

# 2. Обучение

In [9]:
X = df.drop('toxic', axis=1)
y = df['toxic']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=12345)

### LogisticRegression

In [11]:
%%time

lr_pipe = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,3), min_df=3, max_df=0.9, use_idf=1,
               smooth_idf=1, sublinear_tf=1, stop_words=stopwords)),
    ('clf', LogisticRegression(random_state=12345))])

params = {'clf__C': [0.1, 1, 10, 100],
          'clf__class_weight': ['balanced', None]}

lr_grid = GridSearchCV(estimator=lr_pipe, param_grid=params, cv=3, scoring='f1', n_jobs=-1, refit=False)
lr_grid.fit(X_train['text'], y_train)
lr_best_paramms = lr_grid.best_params_

print(lr_best_paramms)
print(lr_grid.best_score_)

{'clf__C': 10, 'clf__class_weight': 'balanced'}
0.7735032698242027
CPU times: user 21min 41s, sys: 2min 44s, total: 24min 25s
Wall time: 24min 29s


### LGBMClassifier

In [12]:
%%time

lgb_pipe = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,3), min_df=3, max_df=0.9, use_idf=1,
               smooth_idf=1, sublinear_tf=1, stop_words=stopwords)),
    ('clf', LGBMClassifier(random_state=12345))])

params = {
  'clf__n_estimators': [200],
  'clf__learning_rate': [0.15, 0.25],
  'clf__max_depth': [8, 10, -1]}

lgb_grid = GridSearchCV(estimator=lgb_pipe, param_grid=params, cv=3, scoring='f1', n_jobs=-1, refit=False)
lgb_grid.fit(X_train['text'], y_train)
lgb_best_params = lgb_grid.best_params_

print(lgb_best_params)
print(lgb_grid.best_score_)

{'clf__learning_rate': 0.25, 'clf__max_depth': -1, 'clf__n_estimators': 200}
0.7675986193392761
CPU times: user 1h 24min 17s, sys: 41.4 s, total: 1h 24min 59s
Wall time: 1h 25min 36s


Подобрали параметры с помощью GridSearchCV с расчетом TF-IDF

In [13]:
vectorize = TfidfVectorizer(ngram_range=(1,3),
               min_df=3, max_df=0.9, use_idf=1,
               smooth_idf=1, sublinear_tf=1, stop_words=stopwords)

In [14]:
X_train = vectorize.fit_transform(X_train['text'])
X_test = vectorize.transform(X_test['text'])

In [15]:
lr_m = LogisticRegression(C=10, class_weight='balanced', random_state=12345)
lr_m.fit(X_train, y_train)

LogisticRegression(C=10, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=12345, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [16]:
lgb_m = LGBMClassifier(learning_rate=0.25, max_depth=-1, n_estimators=200, random_state = 12345)
lgb_m.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.25, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=200, n_jobs=-1, num_leaves=31, objective=None,
               random_state=12345, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [17]:
def scoring(fitted_model):
    test_pred = fitted_model.predict(X_test)
    test_f1 = f1_score(y_test, test_pred)
    
    print('F1 on test: {:.3f}'.format(test_f1))

In [18]:
scoring(lgb_m)

F1 on test: 0.777


In [19]:
scoring(lr_m)

F1 on test: 0.783


# 3. Выводы

Логистическая регрессия выдает результат F1-score: 0.783 на тестовой выборке, это лучший результат.